In [ ]:
# ------------------------------------------------------------------------
#
# TITLE - galaxy_sample.ipynb
# AUTHOR - James Lane
# PROJECT - tng-dfs
#
# ------------------------------------------------------------------------
#
# Docstrings and metadata:
'''Make some plots of the galaxy sample
'''

__author__ = "James Lane"

In [ ]:
### Imports

# Basic
import numpy as np
import sys, os, pdb, copy, glob, subprocess, dill as pickle, h5py

# Matplotlib
import matplotlib as mpl
from matplotlib import pyplot as plt

# Astropy
from astropy import units as apu

# Project-specific
sys.path.insert(0,'../../src/')
from tng_dfs import cutout as pcutout
from tng_dfs import io as pio
from tng_dfs import kinematics as pkin
from tng_dfs import plot as pplot
from tng_dfs import tree as ptree
from tng_dfs import util as putil

### Notebook setup

%matplotlib inline
plt.style.use('../../src/mpl/project.mplstyle') # This must be exactly here
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

In [ ]:
# %load ../../src/nb_modules/nb_setup.txt
# Keywords
cdict = putil.load_config_to_dict()
keywords = ['DATA_DIR','MW_ANALOG_DIR','RO','VO','ZO','LITTLE_H',
            'MW_MASS_RANGE']
data_dir,mw_analog_dir,ro,vo,zo,h,mw_mass_range = \
    putil.parse_config_dict(cdict,keywords)

# MW Analog 
mwsubs,mwsubs_vars = putil.prepare_mwsubs(mw_analog_dir,h=h,
    mw_mass_range=mw_mass_range,return_vars=True,force_mwsubs=False,
    bulge_disk_fraction_cuts=True)

# Figure path
fig_dir = './fig/'
epsen_fig_dir = '/epsen_data/scr/lane/projects/tng-dfs/figs/notebooks/paper/'
os.makedirs(fig_dir,exist_ok=True)
os.makedirs(epsen_fig_dir,exist_ok=True)
show_plots = False

# Load tree data
tree_primary_filename = os.path.join(mw_analog_dir,
    'major_mergers/tree_primaries.pkl')
with open(tree_primary_filename,'rb') as handle: 
    tree_primaries = pickle.load(handle)
tree_major_mergers_filename = os.path.join(mw_analog_dir,
    'major_mergers/tree_major_mergers.pkl')
with open(tree_major_mergers_filename,'rb') as handle:
    tree_major_mergers = pickle.load(handle)
n_mw = len(tree_primaries)

### Plotting Information

In [ ]:
# Column-sized figure width
columnwidth = 244./72.27 # In inches, from pt
# Full-sized figure width
textwidth = 508./72.27 # In inches, from pt

### Make a figure showing some of the galaxies

In [ ]:
indxs = [0,2,3,4,6]

# Keywords
bins = 100
xrange = [-35,35]
yrange = [-35,35]
zrange = [-35,35]
vmin = 6 # Msun/kpc^2
vmax = 9.5 # Msun/kpc^2
label_fs = 8
ticklabel_fs = 8
annotate_fs = 6
annotate_color = 'Black'
cmap = plt.cm.gray

# Plot
fig = plt.figure(figsize=(textwidth,3))
axs = fig.subplots(nrows=2, ncols=len(indxs))

for i in range(len(indxs)):
    # if i > 0: continue

    indx = indxs[i]

    # Get the primary
    primary = tree_primaries[indx]
    z0_sid = primary.subfind_id[0]
    n_snap = len(primary.snapnum)
    primary_filename = primary.get_cutout_filename(mw_analog_dir,
        snapnum=primary.snapnum[0])
    co = pcutout.TNGCutout(primary_filename)
    co.center_and_rectify()

    orbs = co.get_orbs(ptype='stars')
    masses = co.get_masses(ptype='stars').to(apu.Msun).value
    x, y, z = orbs.x().value, orbs.y().value, orbs.z().value

    # Plot face 2d hist and then edge on 2d hist
    N, xedges, yedges = np.histogram2d(x, y, bins=bins, range=[xrange,yrange], 
        weights=masses)
    area = (xedges[1]-xedges[0])*(yedges[1]-yedges[0]) # in kpc
    surfdens = N/area
    img = axs[0,i].imshow(np.log10(surfdens.T), origin='lower', 
        cmap=cmap, vmin=vmin, vmax=vmax, 
        extent=[xedges[0], xedges[-1], yedges[0], yedges[-1]])
    
    N, xedges, zedges = np.histogram2d(x, z, bins=bins, range=[xrange,zrange], 
        weights=masses)
    area = (xedges[1]-xedges[0])*(zedges[1]-zedges[0]) # in kpc
    surfdens = N/area
    img = axs[1,i].imshow(np.log10(surfdens.T), origin='lower', 
        cmap=cmap, vmin=vmin, vmax=vmax, 
        extent=[xedges[0], xedges[-1], zedges[0], zedges[-1]])

    if cmap is plt.cm.gray:
        axs[0,i].set_facecolor('black')
        axs[1,i].set_facecolor('black')
        annotate_color = 'White'
        
    if i == 0:
        axs[1,i].set_ylabel('Z [kpc]', fontsize=label_fs)
        axs[0,i].set_ylabel('Y [kpc]', fontsize=label_fs)
    else:
        axs[0,i].set_yticklabels([])
        axs[1,i].set_yticklabels([])
    axs[1,i].set_xlabel('X [kpc]', fontsize=label_fs)
    axs[0,i].tick_params(axis='both', which='major', labelsize=ticklabel_fs)
    axs[0,i].set_xticks([-30,-15,0,15,30])
    axs[0,i].set_xticklabels([])
    axs[0,i].set_yticks([-30,-15,0,15,30])
    axs[1,i].tick_params(axis='both', which='major', labelsize=ticklabel_fs)
    axs[1,i].set_xticks([-30,-15,0,15,30])
    axs[1,i].set_yticks([-30,-15,0,15,30])
    axs[0,i].annotate(str(z0_sid), xy=(0.05,0.9), xycoords='axes fraction',
        fontsize=annotate_fs, color=annotate_color)

# Create a colorbar on the right size
cbar_ax = fig.add_axes([0.92, 0.19, 0.02, 0.7])
cbar = fig.colorbar(img, cax=cbar_ax)
cbar.set_label(r'$\log_{10} \Sigma_{\star}$ [M$_{\odot}$ kpc$^{-2}$]', 
    fontsize=label_fs)
cbar.ax.tick_params(labelsize=ticklabel_fs)

fig.tight_layout()
fig.subplots_adjust(wspace=0.1, hspace=0.05, right=0.9)
fig.show()
fig.savefig(fig_dir+'galaxy_xyz_projection.pdf', dpi=300, bbox_inches='tight')